_Version log: Considers moving n detectors at a time._

In [22]:
import sys, itertools, random, math, copy, time
import numpy as np
import CS_functions as cs
from tqdm import tqdm
from matplotlib import pyplot as plt
from scipy import fft as spfft

plt.rcParams.update({'font.size':16})
#np.set_printoptions(threshold=sys.maxsize)

In [23]:
# made by Thomas Lux on Stack Overflow
# Return a randomized "range" using a Linear Congruential Generator
# to produce the number sequence. Parameters are the same as for 
# python builtin "range".
#   Memory  -- storage for 8 integers, regardless of parameters.
#   Compute -- at most 2*"maximum" steps required to generate sequence.
#
def random_range(start, stop=None, step=None):
    # Set a default values the same way "range" does.
    if (stop == None): start, stop = 0, start
    if (step == None): step = 1
    # Use a mapping to convert a standard range into the desired range.
    mapping = lambda i: (i*step) + start
    # Compute the number of numbers in this range.
    maximum = (stop - start) // step
    # Seed range with a random integer.
    value = random.randint(0, maximum-1)
    # 
    # Construct an offset, multiplier, and modulus for a linear
    # congruential generator. These generators are cyclic and
    # non-repeating when they maintain the properties:
    # 
    #   1) "modulus" and "offset" are relatively prime.
    #   2) ["multiplier" - 1] is divisible by all prime factors of "modulus".
    #   3) ["multiplier" - 1] is divisible by 4 if "modulus" is divisible by 4. #rule three seems a little arbitrary. Why does it matter and are there any other multiples that we need to watch out for?
    # 
    offset = random.randint(0, maximum-1) * 2 + 1      # Pick a random odd-valued offset.
    #multiplier = 4*(maximum//4) + 1                 # Pick a multiplier 1 greater than a multiple of 4.
    modulus = int(2**math.ceil(math.log2(maximum))) # Pick a modulus just big enough to generate all numbers (power of 2).
    # Track how many random numbers have been returned.
    found = 0
    while found < maximum:
        # If this is a valid value, yield it in generator fashion.
        if value < maximum:
            found += 1
            yield mapping(value)
        # Calculate the next value in the sequence.
        #value = (value*multiplier + offset) % modulus
        value = (value +offset) % modulus #removing the multiplier makes it less random but more reliable for extremely large numbers (>1e13)
    

# made by openai (but debugged by me because robots are not taking over the world anytime soon!)
def find_nth_combination(N, r, idx):
    num_combinations = math.comb(N, r)
    
    if num_combinations <= idx :
        raise ValueError("idx is larger than the total number of combinations")
    
    result = []
    n = 0
    while r > 0:
        num_combinations = math.comb(N -n -1, r - 1)
        if num_combinations <= idx:
            n += 1
            idx -= num_combinations
        else:
            result.append(n)
            n += 1
            r -= 1
        
        if r == 0:
            return tuple(result)
    
    return None


In [24]:
file_number = 15
file_name = "1dmockanderrors{:d}".format(file_number)
#file_name = "240802134128_altered1d"
file_type = ".csv"

optlocs_file = "data\\" + file_name +"_optlocs.csv"
target, uncertainties = cs.open_dataset(file_name, file_type)
total_points = len(target)

In [25]:
################ INITIALISE AND RESET BRUTE FORCE ######################

reduced_points = 6
regularization_coeffient = 1e-3
depth = 1 # How many detectors can the algorithm move at once? DO NOT INCREASE THIS TOO MUCH. It has expontial time complexity.
max_iterations = 20

best_detectors = cs.subsample_1d(total_points, reduced_points, "regular")
best_score = cs.evaluate_score(best_detectors, target, uncertainties, regularization_coeffient)

In [26]:
################# LIMITED BRUTE FORCE ###################

print(math.comb(reduced_points, depth) *math.comb(total_points, depth))

start_time = time.time()

for n in range(max_iterations):
    print(end= "[")
    print(*best_detectors, sep= ", ", end= "] ")
    print(best_score)
    old_detectors = np.copy(best_detectors)

    # pick_detectors = (find_nth_combination(reduced_points, depth, random_index) for random_index in random_range(math.comb(reduced_points, depth))) # checking in a random order is useful if you plan on stopping the algorithm early
    # pick_detectors = (find_nth_combination(reduced_points, depth, random_index) for random_index in range(math.comb(reduced_points, depth))) # higher values of depth will produce better results at the cost of computation time
    pick_detectors = (index for index in range(reduced_points)) # simplest case where only one detector is moved at a time
    
    for moving_detectors in pick_detectors:

        # pick_samples = (find_nth_combination(total_points, depth, random_index) for random_index in range(math.comb(total_points, depth))) # checking in a random order is useful if you plan on stopping the algorithm early
        # pick_samples = (find_nth_combination(total_points, depth, random_index) for random_index in range(math.comb(total_points, depth))) # higher values of depth will produce better results at the cost of computation time
        pick_samples = (index for index in range(total_points) if index not in old_detectors) # simplest case where only one detector is moved at a time. Now checks for duplicates.
        for new_samples in pick_samples:
            detectors = np.copy(old_detectors)
            detectors[np.array(moving_detectors)] = new_samples

            score = cs.evaluate_score(detectors, target, uncertainties, regularization_coeffient)

            if score < best_score:
                best_detectors = np.copy(detectors)
                best_score = np.copy(score)

    if np.all([old_detectors == best_detectors]):
        break


end_time = time.time()

print(end_time - start_time)

cs.append_array_to_csv(best_detectors, optlocs_file)

1200
[0, 40, 80, 119, 159, 199] 9.721031843630236
[86, 40, 80, 119, 159, 199] 8.07729114495846
[86, 40, 80, 92, 159, 199] 5.036153137404374
[86, 40, 80, 92, 159, 128] 3.6207796786464197
[86, 67, 80, 92, 159, 128] 2.302831240441394
[86, 67, 105, 92, 159, 128] 1.9302855464598552
[86, 67, 105, 92, 54, 128] 1.5819109146414745
[98, 67, 105, 92, 54, 128] 1.5572202478539086
[98, 74, 105, 92, 54, 128] 1.4576787830458964
[98, 74, 116, 92, 54, 128] 1.2509235246902781
[98, 73, 116, 92, 54, 128] 1.1964725642876342
55.04244422912598


In [27]:
print(best_score)

1.1964725642876342


In [28]:
print(end="[")
print(*best_detectors, sep= ",", end="]")

[98,73,116,92,54,128]